In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("temp.txt")
df['1'] = 0
df['0'] = df['0'].apply(lambda x:re.sub(r'[^가-힣]+',"",x))
df = df.drop_duplicates(['0'])
df2 = pd.read_csv("save.csv")
df2 = pd.DataFrame(df2['11'])
df2[1] = 1
df2.columns = ['0','1']
temp = pd.concat([df,df2])
temp.to_csv('sentiment.csv')
temp['1'].value_counts()

1    45135
0    32418
Name: 1, dtype: int64

In [ ]:
import pandas as pd
from konlpy.tag import Okt
okt = Okt()
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf.fit(temp['0'])
res = tfidf.transform(temp['0'].values.astype('U'))

In [ ]:
from sklearn.linear_model import  LogisticRegression
sa_lr = LogisticRegression(random_state=0)
sa_lr.fit(res,temp['1'])

df2['1'].value_counts()

In [15]:
from sklearn.model_selection import GridSearchCV
params = {'C':[1,3,3.5,4,4.5,5]}
grid_cv = GridSearchCV(sa_lr, param_grid = params, cv=3, scoring = 'accuracy', verbose=1)
grid_cv.fit(res, temp['1'])

In [18]:
#정확도 테스트
best = grid_cv.best_estimator_
prd = best.predict(res)
from sklearn.metrics import accuracy_score
print('score : ', round(accuracy_score(temp['1'], prd),4))

score :  0.9994


In [28]:
#TEST
st = input('text>>')
st = re.compile(r'[가-힣]+').findall(st)
st = [" ".join(st)]
st_tfidf = tfidf.transform(st)
st_predict = best.predict(st_tfidf)
if st_predict==0:
    print(st, "->> 일반적인 문자")
else:
    print(st, "->> 스미싱 의심문자")


[''] ->> 일반적인 문자


In [38]:
#SAVE TRAIN
import pickle
pickle.dump(res, open("transform.pickle","wb"))
pickle.dump(tfidf, open("tfidf.pickle","wb"))
pickle.dump(grid_cv, open("grid_cv.pickle","wb"))